## Importing Libraries

In [0]:
import os
from google.colab import drive
import tensorflow as tf
import pandas as pd
import numpy as np
import zipfile

In [2]:
import nltk
nltk.download()
from nltk.tokenize import word_tokenize,sent_tokenize

NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> l
Packages:
  [ ] abc................. Australian Broadcasting Commission 2006
  [ ] alpino.............. Alpino Dutch Treebank
  [ ] averaged_perceptron_tagger Averaged Perceptron Tagger
  [ ] averaged_perceptron_tagger_ru Averaged Perceptron Tagger (Russian)
  [ ] basque_grammars..... Grammars for Basque
  [ ] biocreative_ppi..... BioCreAtIvE (Critical Assessment of Information
                           Extraction Systems in Biology)
  [ ] bllip_wsj_no_aux.... BLLIP Parser: WSJ Model
  [ ] book_grammars....... Grammars from NLTK Book
  [ ] brown............... Brown Corpus
  [ ] brown_tei........... Brown Corpus (TEI XML Version)
  [ ] cess_cat............ CESS-CAT Treebank
  [

In [4]:
!pip3 install --upgrade gensim
import gensim

Requirement already up-to-date: gensim in /usr/local/lib/python3.6/dist-packages (3.6.0)


In [5]:
!pip3 install glove_python

    100% |████████████████████████████████| 266kB 9.3MB/s 
  Running setup.py bdist_wheel for glove-python ... - \ | / - \ | done
  Stored in directory: /root/.cache/pip/wheels/88/4b/6d/10c0d2ad32c9d9d68beec9694a6f0b6e83ab1662a90a089a4b
Successfully built glove-python


In [0]:
tf.test.gpu_device_name()

'/device:GPU:0'

### Mounting Drive

In [7]:
drive.mount('/content/gdrive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive/


In [8]:
!ls 'gdrive/My Drive/ASAG/Mohler/'

First_Attempt.ipynb	    Mohler_Dataset_Exploration.ipynb
glove.6B.100d.txt	    papers
glove.6B.100d.txt.word2vec  ShortAnswerGrading_v1.0
Mohler_Dataset_2.ipynb	    ShortAnswerGrading_v2.0


## Loading Data

### Functions

In [0]:
## Global Variables

questions = {}
ref_answer = {}
grades = {}
answers = {}
key=0

In [0]:
def get_blocks(file):
    block_nums = []
    for i in range(0,len(file)):
        if(file[i][0]=='#'):
            block_nums.append(i)
            #print(i)
    global key 
    blocks={}
    for i in range(len(block_nums)-1):
        blocks[key] = [(block_nums[i]+1),(block_nums[i+1]-2)] 
        key = key+1
    blocks[key] = [(block_nums[len(block_nums)-1]+1),(len(file)-2)]  
    return blocks

In [0]:
def extract_info_from_block(block_num,file):
    start = blocks[block_num][0]
    end = blocks[block_num][1]
    temp_grades = {}
    temp_answers = {}
    questions[block_num] = file[start][12:]
    ref_answer[block_num] = file[start+1][9:]    
    m=0
    for i in range(start+3,end+1):
                temp_grades[m] = file[i].split()[0]
                temp_answers[m] = " ".join(x for x in file[i].split()[2:])
                m = m+1
    grades[block_num] = temp_grades
    answers[block_num] = temp_answers   

### Extracting queries, reference_answer, grades and responses

In [12]:
filename = "gdrive/My Drive/ASAG/Mohler/ShortAnswerGrading_v1.0/assign"
for i in range(1,4):#number of text files
    data = open(filename+(str)(i)+".txt","r").readlines()
    blocks = get_blocks(data)
    for key in blocks.keys():
        extract_info_from_block(key,data)
    print("Extracted file:" +filename+(str)(i)+".txt" )    

Extracted file:gdrive/My Drive/ASAG/Mohler/ShortAnswerGrading_v1.0/assign1.txt
Extracted file:gdrive/My Drive/ASAG/Mohler/ShortAnswerGrading_v1.0/assign2.txt
Extracted file:gdrive/My Drive/ASAG/Mohler/ShortAnswerGrading_v1.0/assign3.txt


In [13]:
questions

{0: 'What is the role of a prototype program in problem solving?\n',
 1: 'What stages in the software life cycle are influenced by the testing stage?\n',
 2: 'What are the main advantages associated with object-oriented programming?\n',
 3: 'Where do C++ programs begin to execute?\n',
 4: 'What is a variable?\n',
 5: 'Where are variables declared in a C++ program?\n',
 6: 'What is typically included in a class definition?\n',
 7: ' What is the difference between a data member and a local variable inside a member function?\n',
 8: ' What is the difference between a constructor and a function?\n',
 9: 'When does C++ create a default constructor?\n',
 10: ' How many constructors can be created for a class?\n',
 11: ' What is the difference between a function prototype and a function definition?\n',
 12: 'What does a function signature include?\n',
 13: ' What is the scope of global variables?\n',
 14: ' How does the compiler handle inline functions?\n',
 15: ' What is the main advantage a

In [14]:
ref_answer

{0: ' To simulate the behaviour of portions of the desired software product. \n',
 1: ' The testing stage can influence both the coding stage (phase 5) and the solution refinement stage (phase 7) \n',
 2: ' Abstraction and reusability.\n',
 3: ' At the main function.\n',
 4: ' A location in memory that can store a value.\n',
 5: ' Variables can be declared anywhere in a program. They can be declared inside a function (local variables) or outside the functions (global variables)\n',
 6: '  Data members (attributes) and member functions.\n',
 7: '  Data members can be accessed from any member functions inside the class defintion. Local variables can only be accessed inside the member function that defines them.\n',
 8: ' A constructor is called whenever an object is created, whereas a function needs to be called explicitely. Constructors do not have return type, but functions have to indicate a return type.\n',
 9: ' If no constructor is provided, the compiler provides one by default. If

In [15]:
grades.keys()

dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18])

In [16]:
len(grades.keys())

19

In [0]:
answers

{0: {0: 'High risk problems are address in the prototype program to make sure that the program is feasible. A prototype may also be used to show a company that the software can be possibly programmed.<br><br>',
  1: 'To simulate portions of the desired final product with a quick and easy program that does a small specific job. It is a way to help see what the problem is and how you may solve it in the final project.',
  2: 'A prototype program simulates the behaviors of portions of the desired software product to allow for error checking.',
  3: 'Defined in the Specification phase a prototype stimulates the behavior of portions of the desired software product. Meaning, the role of a prototype is a temporary solution until the program itself is refined to be used extensively in problem solving.',
  4: 'It is used to let the users have a first idea of the completed program and allow the clients to evaluate the program. This can generate much feedback including software specifications and

## Converting to Glove

### Getting pre-trained Glove Embeddings

In [0]:
# from gensim.scripts.glove2word2vec import glove2word2vec
# glove_input_file = 'gdrive/My Drive/ASAG/Mohler/glove.6B.100d.txt'
# word2vec_output_file = 'gdrive/My Drive/ASAG/Mohler/glove.6B.100d.txt.word2vec'
# glove2word2vec(glove_input_file, word2vec_output_file)

In [0]:

# from gensim.models import KeyedVectors
# # load the Stanford GloVe model
# filename = 'gdrive/My Drive/ASAG/Mohler/glove.6B.100d.txt.word2vec'
# model = KeyedVectors.load_word2vec_format(filename, binary=False)
# # calculate: (king - man) + woman = ?
# result = model.most_similar(positive=['woman', 'king'], negative=['man'], topn=1)
# print(result)

In [0]:
# model.get_vector("man")

### Tokenzing words

In [0]:
qs=questions
ra=ref_answer
a=answers
g=grades

for q in questions.keys():
  qs[q] = word_tokenize(questions[q].lower())
  ra[q] = word_tokenize(ref_answer[q].lower())
for k in answers.keys():
  for response in answers[k]:
    a[k][response] = word_tokenize(answers[k][response].lower().replace("<br>",""))  

### Training our own Glove model

In [0]:

common_list=[]
for q in qs.keys():
  common_list.append(qs[q])
for ans in ra.keys():
  common_list.append(ra[ans])


In [0]:
for k in a.keys():
  for response in a[k]:
    common_list.append(a[k][response])

In [0]:
common_list[600]

['they',
 'are',
 'both',
 'based',
 'on',
 'a',
 'control',
 'statement',
 ',',
 'both',
 'involve',
 'repetition',
 ',',
 'both',
 'involve',
 'a',
 'termination',
 'case',
 ',',
 'both',
 'graduatlly',
 'approach',
 'that',
 'termination',
 'case',
 ',',
 'and',
 'both',
 'can',
 'occur',
 'infinitely']

In [0]:
word = []
for i in range(len(common_list)):
  for j in range(len(common_list[i])):
    if(common_list[i][j] not in word):
      word.append(common_list[i][j])

In [0]:
word

['what',
 'is',
 'the',
 'role',
 'of',
 'a',
 'prototype',
 'program',
 'in',
 'problem',
 'solving',
 '?',
 'stages',
 'software',
 'life',
 'cycle',
 'are',
 'influenced',
 'by',
 'testing',
 'stage',
 'main',
 'advantages',
 'associated',
 'with',
 'object-oriented',
 'programming',
 'where',
 'do',
 'c++',
 'programs',
 'begin',
 'to',
 'execute',
 'variable',
 'variables',
 'declared',
 'typically',
 'included',
 'class',
 'definition',
 'difference',
 'between',
 'data',
 'member',
 'and',
 'local',
 'inside',
 'function',
 'constructor',
 'when',
 'does',
 'create',
 'default',
 'how',
 'many',
 'constructors',
 'can',
 'be',
 'created',
 'for',
 'signature',
 'include',
 'scope',
 'global',
 'compiler',
 'handle',
 'inline',
 'functions',
 'advantage',
 'arguments',
 'that',
 'passed',
 'reference',
 'overloaded',
 'differentiated',
 'defining',
 'recursive',
 ',',
 'possible',
 'causes',
 'infinite',
 'recursion',
 'similarities',
 'iteration',
 'simulate',
 'behaviour',
 'po

In [0]:
len(word)

1308

In [0]:
v_len = int(np.ceil(np.sqrt(np.sqrt(len(word)))))

In [0]:
lines = common_list
#importing the glove library
from glove import Corpus, Glove

# creating a corpus object
corpus = Corpus() 

#training the corpus to generate the co occurence matrix which is used in GloVe
#window is the distance between two words algo should consider to find some relationship between them
corpus.fit(lines, window=10)

#creating a Glove object which will use the matrix created in the above lines to create embeddings
#We can set the learning rate as it uses Gradient Descent and number of components
#No_components is the dimensions of output vector
glove = Glove(no_components=v_len, learning_rate=0.05)
 
glove.fit(corpus.matrix, epochs=1500, no_threads=4, verbose=True)
glove.add_dictionary(corpus.dictionary)
glove.save('glove.model')

Performing 1500 training epochs with 4 threads
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19
Epoch 20
Epoch 21
Epoch 22
Epoch 23
Epoch 24
Epoch 25
Epoch 26
Epoch 27
Epoch 28
Epoch 29
Epoch 30
Epoch 31
Epoch 32
Epoch 33
Epoch 34
Epoch 35
Epoch 36
Epoch 37
Epoch 38
Epoch 39
Epoch 40
Epoch 41
Epoch 42
Epoch 43
Epoch 44
Epoch 45
Epoch 46
Epoch 47
Epoch 48
Epoch 49
Epoch 50
Epoch 51
Epoch 52
Epoch 53
Epoch 54
Epoch 55
Epoch 56
Epoch 57
Epoch 58
Epoch 59
Epoch 60
Epoch 61
Epoch 62
Epoch 63
Epoch 64
Epoch 65
Epoch 66
Epoch 67
Epoch 68
Epoch 69
Epoch 70
Epoch 71
Epoch 72
Epoch 73
Epoch 74
Epoch 75
Epoch 76
Epoch 77
Epoch 78
Epoch 79
Epoch 80
Epoch 81
Epoch 82
Epoch 83
Epoch 84
Epoch 85
Epoch 86
Epoch 87
Epoch 88
Epoch 89
Epoch 90
Epoch 91
Epoch 92
Epoch 93
Epoch 94
Epoch 95
Epoch 96
Epoch 97
Epoch 98
Epoch 99
Epoch 100
Epoch 101
Epoch 102
Epoch 103
Epoch 104
Epoch 105
Epo

In [0]:
print (glove.word_vectors[glove.dictionary[qs[0][0]]])

[-0.07941861  0.4468406   0.04821956 -0.16253219 -0.44233967  0.63009201
 -0.21562715]


In [0]:
qsv=qs
rav=ra
av=a

for q in qs.keys():
  for i in range(len(qs[q])):
    qsv[q][i] = glove.word_vectors[glove.dictionary[ qs[q][i] ]]
  for j in range(len(rav[q])):  
    rav[q][j] = glove.word_vectors[glove.dictionary[ ra[q][j] ]]
for k in a.keys():
  for response in a[k]:
    for p in range(len(a[k][response])):
      av[k][response][p] = glove.word_vectors[glove.dictionary[ a[k][response][p] ]]  


In [0]:
max_ql = 0
max_rl = 0
max_al = 0
for q in qsv.keys():
  if len(qsv[q]) > max_ql:
    max_ql = len(qsv[q])

for r in rav.keys():
  if len(rav[r]) > max_rl:
    max_rl = len(rav[r])

for a in av.keys():
  for resp in av[a]:
    if len(av[a][resp]) > max_al:
      max_al = len(av[a][resp])
print ("max Lengths  : " , max_ql, max_rl, max_al)

max Lengths  :  17 37 193


In [0]:
def padding(vect, max_len, v_len):
  diff = max_len - len(vect)
  to_append = np.zeros((diff,v_len),dtype=np.float64)
  return np.concatenate((vect,to_append),axis=0)

In [0]:
for q in qsv.keys():
  qsv[q] = padding(qsv[q], max_ql, v_len)
for r in rav.keys():
  rav[r] = padding(rav[r], max_rl, v_len)
for a in av.keys():
  for resp in av[a]:
    av[a][resp] = padding(av[a][resp], max_al, v_len)  

## Model

In [0]:
import keras.layers as L
import keras.models as M
import keras.optimizers as O
import keras.backend as K

Using TensorFlow backend.


In [0]:
q_inp = L.Input(shape=(max_ql,v_len))
r_inp = L.Input(shape=(max_rl,v_len))
a_inp = L.Input(shape=(max_al,v_len))

q_cnn = L.Conv1D(256,3)(q_inp)
q_cnn = L.MaxPool1D(2)(q_cnn)
q_cnn = L.BatchNormalization()(q_cnn)
q_cnn = L.Conv1D(128,3)(q_cnn)
q_cnn = L.MaxPool1D(2)(q_cnn)
q_cnn = L.BatchNormalization()(q_cnn)

q_rnn = L.LSTM(256,recurrent_dropout=0.2,return_sequences=True,unroll=True)(q_cnn)
# q_rnn = L.LSTM(256,recurrent_dropout=0.2,return_sequences=True,unroll=True)(q_rnn)
# q_rnn = L.LSTM(256,recurrent_dropout=0.2,return_sequences=True,unroll=True)(q_rnn)

q_avg = L.Lambda(lambda y: K.mean(y, axis=1))(q_rnn)
# q_avg = L.Lambda(lambda y: K.squeeze(y, axis=1))(q_avg)

r_cnn = L.Conv1D(256,3)(r_inp)
r_cnn = L.MaxPool1D(2)(r_cnn)
r_cnn = L.BatchNormalization()(r_cnn)
r_cnn = L.Conv1D(128,3)(r_cnn)
r_cnn = L.MaxPool1D(2)(r_cnn)
r_cnn = L.BatchNormalization()(r_cnn)

r_rnn = L.LSTM(256,recurrent_dropout=0.2,return_sequences=True,unroll=True)(r_cnn)
# r_rnn = L.LSTM(256,recurrent_dropout=0.2,return_sequences=True,unroll=True)(r_rnn)
# r_rnn = L.LSTM(256,recurrent_dropout=0.2,return_sequences=True,unroll=True)(r_rnn)

r_avg = L.Lambda(lambda y: K.mean(y, axis=1))(r_rnn)

#for answers
a_cnn = L.Conv1D(256,3)(a_inp)
a_cnn = L.MaxPool1D(2)(a_cnn)
a_cnn = L.BatchNormalization()(a_cnn)
a_cnn = L.Conv1D(128,3)(a_cnn)
a_cnn = L.MaxPool1D(2)(a_cnn)
a_cnn = L.BatchNormalization()(a_cnn)

a_rnn = L.LSTM(256,recurrent_dropout=0.2,return_sequences=True,unroll=True)(a_cnn)
# a_rnn = L.LSTM(256,recurrent_dropout=0.2,return_sequences=True,unroll=True)(a_rnn)
# a_rnn = L.LSTM(256,recurrent_dropout=0.2,return_sequences=True,unroll=True)(a_rnn)

a_avg = L.Lambda(lambda y: K.mean(y, axis=1))(a_rnn)
# r_avg = L.Lambda(lambda y: K.squeeze(y, axis=1))(r_avg)

f_inp = L.Concatenate(axis=1)([q_avg,r_avg,a_avg])

f_out = L.Dense(1024,activation='relu')(f_inp)
f_out = L.Dropout(0.3)(f_out)
f_out = L.Dense(1024,activation='relu')(f_out)
f_out = L.Dropout(0.3)(f_out)
f_out = L.Dense(1024,activation='relu')(f_out)
f_out = L.Dropout(0.3)(f_out)
f_out = L.Dense(1,activation='relu')(f_out)

model = M.Model(inputs=[q_inp,r_inp,a_inp],outputs=[f_out],name='Binary Classify')
optim = O.Adam(lr=0.001,decay=0.9)
model.compile(optimizer=optim,loss='mse',metrics=['mae'])

In [0]:
q_inp = L.Input(shape=(max_ql,v_len))
r_inp = L.Input(shape=(max_rl,v_len))
a_inp = L.Input(shape=(max_al,v_len))

inps = L.Concatenate(axis=1)([q_inp,r_inp,a_inp])
inps = L.Lambda(lambda y: K.mean(inps,axis=1))(inps)

d = L.Dense(1024, activation='relu')(inps)
d = L.Dense(1024, activation='relu')(d)
d = L.Dense(512, activation='relu')(d)
f_out = L.Dense(1, activation='relu')(d)

model = M.Model(inputs=[q_inp,r_inp,a_inp],outputs=[f_out],name='Binary Classify')
optim = O.Adam(lr=0.001,decay=0.9)
model.compile(optimizer=optim,loss='mse',metrics=['mae'])

In [0]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_10 (InputLayer)           (None, 17, 7)        0                                            
__________________________________________________________________________________________________
input_11 (InputLayer)           (None, 37, 7)        0                                            
__________________________________________________________________________________________________
input_12 (InputLayer)           (None, 193, 7)       0                                            
__________________________________________________________________________________________________
concatenate_4 (Concatenate)     (None, 247, 7)       0           input_10[0][0]                   
                                                                 input_11[0][0]                   
          

Model, features, data.. Run for all datasets and then decide which one to modify

In [0]:
g_un = []
for i in grades.keys():
  for j in grades[i].keys():
    if(grades[i][j] not in g_un):
      g_un.append(grades[i][j])

In [0]:
len(g_un)

10

In [0]:
split = int(0.6*len(qsv.keys()))

In [0]:
def train_test_split(qsv, rav, av, grades, split):
  train_qsv, test_qsv, train_rav, test_rav, train_av, test_av, train_grade, test_grade = [],[],[],[],[],[],[],[]
  for i in range(split):
    rep = len(av[i])
    for k in range(rep):
      train_qsv.append(qsv[i])
      train_rav.append(rav[i])
    for j in av[i].keys():
      train_av.append(av[i][j])
      train_grade.append(grades[i][j])
      
  for i in range(split,len(qsv)):
    rep = len(av[i].keys())
    for k in range(rep):
      test_qsv.append(qsv[i])
      test_rav.append(rav[i])
    for j in av[i].keys():
      test_av.append(av[i][j])
      test_grade.append(grades[i][j])

  return np.array(train_qsv), np.array(train_rav), np.array(train_av), np.array(train_grade), np.array(test_qsv), np.array(test_rav), np.array(test_av), np.array(test_grade)

In [0]:
train_qsv, train_rav, train_av, train_grade,test_qsv, test_rav, test_av, test_grade = train_test_split(qsv, rav, av, grades, split)

In [0]:
train_qsv.shape, train_rav.shape, train_av.shape, train_grade.shape

((324, 17, 7), (324, 37, 7), (324, 193, 7), (324,))

In [0]:
test_qsv.shape, test_rav.shape, test_av.shape, test_grade.shape

((247, 17, 7), (247, 37, 7), (247, 193, 7), (247,))

In [0]:
model.fit(x = [train_qsv, train_rav, train_av], y = [train_grade], validation_data=([test_qsv, test_rav, test_av],[test_grade]), batch_size=128, epochs=500)

Train on 324 samples, validate on 247 samples
Epoch 1/500
324/324 [==============================] - 1s 2ms/step - loss: 2.1157 - mean_absolute_error: 1.2375 - val_loss: 1.8777 - val_mean_absolute_error: 1.1449
Epoch 2/500
324/324 [==============================] - 0s 113us/step - loss: 2.1154 - mean_absolute_error: 1.2374 - val_loss: 1.8773 - val_mean_absolute_error: 1.1447
Epoch 3/500
324/324 [==============================] - 0s 106us/step - loss: 2.1150 - mean_absolute_error: 1.2373 - val_loss: 1.8769 - val_mean_absolute_error: 1.1446
Epoch 4/500
324/324 [==============================] - 0s 106us/step - loss: 2.1148 - mean_absolute_error: 1.2372 - val_loss: 1.8765 - val_mean_absolute_error: 1.1444
Epoch 5/500
324/324 [==============================] - 0s 144us/step - loss: 2.1144 - mean_absolute_error: 1.2371 - val_loss: 1.8761 - val_mean_absolute_error: 1.1443
Epoch 6/500
324/324 [==============================] - 0s 107us/step - loss: 2.1142 - mean_absolute_error: 1.2370 - val_l

In [0]:
model.evaluate(x=[test_qsv, test_rav, test_av], y=[test_grade], batch_size=128)

247/247 [==============================] - 0s 65us/step


[1.7401215938421397, 1.1031093650501267]

In [0]:
pred = model.predict(x=[test_qsv, test_rav, test_av])

In [0]:
pred1 = np.round(pred*2) / 2

In [0]:
p = pd.DataFrame(np.column_stack((pred1,test_grade)),columns=['Pred','Truth'])

In [0]:
p

,Pred,Truth
0,5.5,4
1,10.0,2.5
2,5.5,5
3,5.0,3
4,5.5,5
5,5.0,4.5
6,5.0,4.5
7,5.5,4.5
8,5.5,4
9,5.0,4.5
